<a href="https://colab.research.google.com/github/CarlosSilva59/Projeto-Alura-Google-aula-04/blob/main/Projeto-Alura-Google-aula-04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código em desenvolvimento

In [ ]:
import streamlit as st
import pandas as pd
from google.cloud import storage
from io import BytesIO
import base64

def salvar_imagem_no_cloud_storage(imagem, tipo_doacao, quantidade, bloco_armazenamento):
    """
    Salva a imagem no Cloud Storage e retorna a URL pública da imagem.

    Args:
        imagem (UploadedFile): Objeto de imagem carregada pelo usuário.
        tipo_doacao (str): Tipo de doação.
        quantidade (int): Quantidade de doação.
        bloco_armazenamento (str): Bloco de armazenamento.

    Returns:
        str: URL pública da imagem salva no Cloud Storage.
    """
    try:
        # Convertendo a imagem para base64
        buffer = BytesIO()
        imagem.readinto(buffer)
        encoded_image = base64.b64encode(buffer.getvalue()).decode()

        # Salvando a imagem no Cloud Storage
        storage_client = storage.Client()
        bucket = storage_client.get_bucket("seu-bucket-do-google-cloud-storage")
        image_name = f"{tipo_doacao}-{quantidade}-{bloco_armazenamento}.jpg"
        blob = bucket.blob(image_name)
        blob.upload_from_string(encoded_image, options={"api_key": ""}) # preencher a API KEY

        # Retornando a URL pública da imagem
        return blob.public_url_for(signed_url_version='v2')

    except Exception as e:
        print(f"Erro ao salvar imagem: {e}")
        return ""

# Configurando a página do Streamlit
st.title("Doações para Desalojados no Rio Grande do Sul")
st.markdown("Ajude-nos a auxiliar os desalojados pelas fortes chuvas no Rio Grande do Sul! Doe materiais de primeira necessidade e contribua para a reconstrução de vidas.")

# Criando um formulário para receber as informações da doação
formulario = st.form("Doação")
tipo_doacao = formulario.selectbox("Tipo de Doação", ["Perecível", "Água", "Colchões", "Roupas", "Outros"])
quantidade = formulario.number_input("Quantidade", min_value=1)
bloco_armazenamento = formulario.selectbox("Bloco de Armazenamento", ["A", "B", "C", "D", "E"])
imagem = formulario.file_uploader("Imagem da Doação (opcional)")
submit_button = formulario.submit_button("Doar")

# Processando as informações da doação
if submit_button:
    try:
        # Validando dados de entrada
        if quantidade <= 0:
            raise ValueError("Quantidade de doação deve ser maior que zero.")

        # Salvando a imagem no Cloud Storage (se houver)
        imagem_url = ""
        if imagem:
            imagem_url = salvar_imagem_no_cloud_storage(imagem, tipo_doacao, quantidade, bloco_armazenamento)
            if not imagem_url:
                st.error("Erro ao salvar imagem. Tente novamente.")
                raise Exception("Erro ao salvar imagem")  # Re-raise exception for further handling

        # Atualizando a tabela de doações
        doacoes_df = pd.read_csv("doacoes.csv", index_col=0)
        nova_doacao = {"Tipo": tipo_doacao, "Quantidade": quantidade, "Bloco de Armazenamento": bloco_armazenamento, "Imagem URL": imagem_url}
        doacoes_df = doacoes_df.append(nova_doacao, ignore_index=True)
        doacoes_df.to_csv("doacoes.csv")
        st.success("Doação registrada com sucesso!")

    except Exception as e:
        st.error(f"Ocorreu um erro durante o processamento da doação: {e}")